# Per-LS Data Exploration Tools Development Notebook

This notebook is intended to provide essential tools to explore per-LS information fetched from DIALS.

## Setup
### DIALS

In [ ]:
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import math
import seaborn as sns
import json
import os

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)

creds = Credentials.from_creds_file()
dials = Dials(creds)

### OMS API

In [ ]:
with open("../clientid.json", "r") as file:
    secrets = json.load(file)

os.environ["API_CLIENT_ID"] = secrets["API_CLIENT_ID"]
os.environ["API_CLIENT_SECRET"] = secrets["API_CLIENT_SECRET"]

import oms

oms_fetch = oms.oms_fetch()

## DQMExplore Class Definition (WIP)

In [ ]:
class dqmexplorer:
    
    def __init__(self) -> None:
        self._ME_names: dict = {"1D": [], "2D": []}
        self._data_df: pd.DataFrame | None = None 
        self._runnbs: list[int|float] | None = None 
        self._avail1DMEs = []
        self._avail2DMEs = []
    
    def set_MEs(self, MEs1D: list | None = None, MEs2D: list | None = None) -> None:
        if MEs1D:
            self._ME_names["1D"] = [ME for ME in MEs1D if ME in self._avail1DMEs] 
        if MEs2D:
            self._ME_names["2D"] = [ME for ME in MEs2D if ME in self._avail1DMEs]

    def get_MEs(self, type: float | None = None) -> list | dict:
        if type:
            if type in ["1D", "2D"]:
                return (self._ME_names["type"]) 
            else:
                raise ValueError("Unknown ME type.")
        else:
            return self._ME_names

    def set_runs(self, runnbs: int | float | list[int|float]) -> None:
        if isinstance(runnbs, (int, float)):
            runnbs = [runnbs]
        
        self._runnbs = runnbs

    # def (self):
        


## Plotting function definitions (WIP)

In [ ]:
def plotinteractive1D(data: np.array, bin_locs: list[float], title: str, x_label: str, y_label: str, plot: bool = True) -> plotly.graph_objs._figure.Figure | None:
    """
    Data assumed to be in order of LS.
    """
    fig = go.Figure()

    max_y = data.max()

    fig = go.Figure(
        data=[
            go.Bar(
                x=bin_locs,
                y=data[0,:],
                visible=True,
                width=np.diff(bin_locs)
            )
        ],
    )

    # Building steps
    steps = []
    for i in range(len(data)):
        step = {
            "method": "restyle",
            "args": [{"y": [data[i, :]]}],
            "label": str(i+1),
        }
        steps.append(step)

    # Building slider using steps defined above
    sliders = [
        {
            "active": 0,
            "currentvalue": {"prefix": "LS: "},
            "pad": {"t": 50},
            "steps": steps,
        }
    ]

    # Passing all of this into the figure
    fig.update_layout(
        sliders=sliders,
        title=title,
        xaxis={
            "title": x_label
        },
        yaxis={
            "range": [0, max_y + 10],
            "title": y_label
        },
        bargap=0,
    )

    if plot:
        fig.show()
    else:
        return fig

In [ ]:
def plotinteractive2D(data: np.array, title: str, x_label: str, y_label: str, plot: bool = True) -> plotly.graph_objs._figure.Figure | None:
    fig = go.Figure()

    fig.add_trace(
        go.Heatmap(
            z=data[0], 
            colorscale="Viridis",
            zmin=data.min(),
            zmax=data.max()
        )
    )

    # Building steps
    steps = []
    for i in range(len(data)):
        step = {
            "method": "restyle",
            "args": [{"z": [data[i]]}],
            "label": str(i + 1)
        }
        steps.append(step)

    sliders = [
        {
            "active": 0,
            "currentvalue": {"prefix": "LS: "},
            "pad": {"t": 10},
            "steps": steps,
        }
    ]

    fig.update_layout(
        sliders=sliders,
        title_text = title,
        title_font = {"size": 24},
        xaxis={
            "title": x_label
        },
        yaxis={
            "title": y_label
        },
        height=750,
        width=750,
    )

    if plot:
        fig.show()
    else:
        return fig

In [ ]:
def multiplotinteractive1D(datas: list[np.array], bin_locs: list[np.array], titles: list[str], x_labels: str, y_labels: str) -> plotly.graph_objs._figure.Figure | None:
    num_mes = len(titles)
    
    if num_mes != len(datas):
        raise ValueError("Number of titles does not match number of datasets given.")

    num_rows = num_mes - num_mes//2
    num_cols = num_mes//2
    num_lss = len(datas[0])

    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles=titles,
        vertical_spacing=0.15,
        horizontal_spacing=0.05
    )

    # Adding the first LS
    for i, ME in enumerate(titles):
        fig.add_trace(go.Bar(x=bin_locs[i], y=datas[i][0], name=ME), row=i//2 + 1, col=i%2 + 1)

    # Making the steps to update to the rest of the LSs
    steps = []
    for i in range(num_lss):
        step = {
            'method': 'restyle',
            'args': [
                {'y': [datas[j][i,:] for j in range(len(datas))]},
                np.arange(len(datas))  # Indices of the traces to modify
            ],
            'label': f'LS {i+1}'
        }
        steps.append(step)
        
    sliders = [{
        'active': 0,
        'currentvalue': {'prefix': 'LS: '},
        'pad': {'t': 50},
        'steps': steps
    }]

    # Adding elements and updating layout
    fig.update_layout(
        sliders=sliders,
        # title="PX Barrel Charge",
        # xaxis_title="Charge (e)",
        # yaxis_title="Count",
        bargap=0,
        showlegend=False,
        width=1600,
        height=900
    )

    for i in range(num_mes):
        max_y = datas[i].max()
        fig.update_yaxes(title_text="Count", range=[0, max_y + 10], row=(i//2) + 1, col=(i%2) + 1)

    fig.show()

In [ ]:
def plot1dheatmap():
    pass

In [ ]:
def multiplot1dheatmap():
    pass

In [ ]:
def multiplotinteractive2D():
    pass

In [ ]:
def multiplotinteractive():
    pass

### Other Helper Functions

In [ ]:
def extractdata(queryrslt: cmsdials.clients.h2d.models.PaginatedLumisectionHistogram2DList, rtn_df=False) -> np.array:
    data_df = pd.DataFrame(queryrslt.dict()["results"])
    data_df.sort_values("ls_number", inplace=True)

    if len(data_df["me"].unique()) == 1:
        data_arr = data_df["data"].to_numpy(dtype=np.ndarray)
        data_arr = np.array([np.array(x) for x in data_arr])

    else:
        data_arr = []
        for me in data_df["me"].unique():
            me_arr = data_df[data_df["me"] == me]["data"].to_numpy(dtype=np.ndarray)
            me_arr = np.array([np.array(x) for x in me_arr])
            data_arr.append(me_arr)
        data_arr = np.array(data_arr)

    if rtn_df:
        return (data_arr, data_df)
    else:
        return data_arr

## Interactive 1D Monitoring Elements
### One histogram

Fetching data

In [ ]:
LumisectionHistogram1DFilters??

In [ ]:
# runnb = 380030
# runnb = 378239
# runnb = 378994 # PX Timing scan
# runnb = 378981 # PX Bias scan
runnb = 380237

data = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        me="PixelPhase1/Tracks/PXBarrel/charge_PXLayer_1",
        dataset__regex="/ZeroBias/",
        # dataset="/ZeroBias/Run2024B-PromptReco-v1/DQMIO",
        run_number=runnb
    ),
    # max_pages=200
)

data_arr, data_df = extractdata(queryrslt=data, rtn_df=True)

In [ ]:
data_df.head(3)

Getting information about data

In [ ]:
# Extracting histogram data
LSs = data_df["ls_number"].unique()
ME = data_df["me"][0]
num_bins = int(data_df["x_bin"][0])

x_min = data_df["x_min"][0]
x_max = data_df["x_max"][0]
x_bin = data_df["x_bin"][0]
bin_locs = np.linspace(x_min, x_max, num_bins)

Plotting

In [ ]:
plotinteractive1D(data=data_arr, bin_locs=bin_locs, title=ME, x_label="Charge (e)", y_label="Count", plot=False)

### Multiple Histograms

Getting the data

In [ ]:
runnb = 380237

data = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        me__regex="PixelPhase1/Tracks/PXBarrel/charge_PXLayer_",
        dataset__regex="/ZeroBias/",
        run_number=runnb
    ),
)

data_arr, data_df = extractdata(queryrslt=data, rtn_df=True)

In [ ]:
data_df.head(8)

In [ ]:
# Extracting histogram data
MEs = data_df["me"].unique()
bin_locs = []
x_mins = []
x_maxs = []
num_bins = []

for i, me in enumerate(MEs):
    x_mins.append(data_df[data_df["me"] == me]["x_min"][0])
    x_maxs.append(data_df[data_df["me"] == me]["x_max"][0])
    num_bins.append(data_df[data_df["me"] == me]["x_bin"][0])
    print(num_bins)
    # bin_locs.append(np.linspace(x_mins[i], x_maxs[i], num_bins[i]))




# LSs = data2d_df["ls_number"].unique()
# ME = data2d_df["me"][0]
# num_xbins = int(data2d_df["x_bin"][0])
# num_ybins = int(data2d_df["y_bin"][0])

# x_min = data2d_df["x_min"][0]
# x_max = data2d_df["x_max"][0]
# x_bin = data2d_df["x_bin"][0]
# bin_xlocs = np.linspace(x_min, x_max, num_bins)

# y_min = data2d_df["y_min"][0]
# y_max = data2d_df["y_max"][0]
# y_bin = data2d_df["y_bin"][0]
# bin_ylocs = np.linspace(y_min, y_max, num_ybins)

In [ ]:
multiplotinteractive1D(datas=data_arr, bin_locs = )

## Interactive 2D Monitoring Elements

In [ ]:
filter2d = LumisectionHistogram2DFilters(
    run_number=runnb,
    dataset__regex="/ZeroBias/",
    me="PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_1"
)

data2d_queryrslt = dials.h2d.list_all(
    filter2d
)

data2d_arr, data2d_df = extractdata(queryrslt=data2d_queryrslt, rtn_df=True)

In [ ]:
data2d_df.head(3)

Getting information about data

In [ ]:
# Extracting histogram data
LSs = data2d_df["ls_number"].unique()
ME = data2d_df["me"][0]
num_xbins = int(data2d_df["x_bin"][0])
num_ybins = int(data2d_df["y_bin"][0])

x_min = data2d_df["x_min"][0]
x_max = data2d_df["x_max"][0]
x_bin = data2d_df["x_bin"][0]
bin_xlocs = np.linspace(x_min, x_max, num_bins)

y_min = data2d_df["y_min"][0]
y_max = data2d_df["y_max"][0]
y_bin = data2d_df["y_bin"][0]
bin_ylocs = np.linspace(y_min, y_max, num_ybins)

In [ ]:
plotinteractive2D(data=data2d_arr, title="PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_1", x_label="z", y_label="phi")

---
## OLD CODE
Whatever is useful here will be added on above

In [ ]:
data_lyr1 = data_df[data_df["me"]=="PixelPhase1/Tracks/PXBarrel/charge_PXLayer_1"]

histbins = data_lyr1["data"].to_numpy(dtype=np.ndarray)
histbins = np.array([np.array(x) for x in histbins])

y_min = histbins.min()
y_max = histbins.max()

In [ ]:
fig = go.Figure()

max_y = histbins.max()

fig = go.Figure(
    data=[
        go.Bar(
            x=bin_locs,
            y=histbins[0,:],
            visible=True,
            width=np.diff(bin_locs)
        )
    ],
)

# Building steps
steps = []
for i in range(len(histbins)):
    step = {
        "method": "restyle",
        "args": [{"y": [histbins[i, :]]}],
        "label": str(i+1),
    }
    steps.append(step)

# Building slider using steps defined above
sliders = [
    {
        "active": 0,
        "currentvalue": {"prefix": "LS: "},
        "pad": {"t": 50},
        "steps": steps,
    }
]

# Passing all of this into the figure
fig.update_layout(
    sliders=sliders,
    title=MEs[0],
    xaxis={
        "title": "Charge (e)"
    },
    yaxis={
        "range": [0, max_y + 10],
        "title": "Count"
    },
    bargap=0,
)

fig.show()

In [ ]:
data_lyrs = []

for i in range(0, 4):
    data_onelyr = data_df[data_df["me"]==f"PixelPhase1/Tracks/PXBarrel/charge_PXLayer_{i+1}"]
    data_onelyr = data_onelyr["data"].to_numpy(dtype=np.ndarray)
    data_onelyr = np.array([np.array(x) for x in data_onelyr])
    data_lyrs.append(data_onelyr)
    
num_ls = len(data_lyrs[0])

In [ ]:
fig, axs = plt.subplots(2, 2, dpi=200)
axs = axs.flatten()

for i, ME in enumerate(MEs):
    sns.heatmap(data_lyrs[i], ax=axs[i])
    axs[i].set_title(ME, fontsize=5)
    
axs = axs.reshape(2,2)

fig.subplots_adjust(hspace=0.5, wspace=0.5)

plt.show()

In [ ]:
go.Heatmap??

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=MEs)

for i, data in enumerate(data_lyrs):
    row = (i // 2) + 1
    col = (i % 2) + 1
    fig.add_trace(
        go.Heatmap(z=data, x=bin_locs, colorscale="Viridis"),
        row=row, col=col,
    )

fig.update_layout(
    title_text = "PX Barrel Charge",
    title_font={"size": 24},
    height=1100,
    width=1100,
    annotations = [
        {"text": ME, "font": {"size": 14}} for ME in MEs]
)

fig.update_yaxes(autorange="reversed")

fig.show()

In [ ]:
make_subplots??

In [ ]:
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=MEs,
    vertical_spacing=0.15,
    horizontal_spacing=0.05
)

# Adding the first LS
for i, ME in enumerate(MEs):
    fig.add_trace(go.Bar(x=bin_locs, y=data_lyrs[i][0], name=ME), row=i//2 + 1, col=i%2 + 1)

# Making the steps to update to the rest of the LSs
steps = []
for i in range(num_ls):
    step = {
        'method': 'restyle',
        'args': [
            {'y': [data_lyrs[j][i,:] for j in range(len(data_lyrs))]},
            np.arange(len(data_lyrs))  # Indices of the traces to modify
        ],
        'label': f'LS {i+1}'
    }
    steps.append(step)
    
sliders = [{
    'active': 0,
    'currentvalue': {'prefix': 'Lumisection: '},
    'pad': {'t': 50},
    'steps': steps
}]

# Adding elements and updating layout
fig.update_layout(
    sliders=sliders,
    title="PX Barrel Charge",
    xaxis_title="Charge (e)",
    yaxis_title="Count",
    bargap=0,
    showlegend=False,
    width=1600,
    height=900
)

for i in range(4):
    fig.update_yaxes(title_text="Count", range=[0, max_y + 10], row=(i//2) + 1, col=(i%2) + 1)

fig.show()

## 2D Histograms